# **Exploratory Data Analysis**

In this notebook, we will explore the nature of our data, first by using the skin cancer dataset from kaggle.

In [1]:
import kaggle

In [2]:
# !kaggle datasets download -d kmader/skin-cancer-mnist-ham10000 -p ../data/raw --unzip -o -q